### **Tensorflow3강 Evaluating&Model Save**
설계한 model의 성능을 평가하고, label값과 비교한다.
설계한 model의 학습된 weight와 bias값을 파일 형태로 저장한다.

### **check point**
weight값과 model의 구성을 알고 있으면, 어떤 input이 들어왔을 때 prediction을 할 수 있다.  
물론! model의 형태가 다르면 안된다.  

test model : gradient-decent, loss 등 train에 관한 모든 logic이 제거가 된 model  
test model에 train된 weight를 대입시키려면 **check point**라는 파일을 만들어 weight 정보를 쓴다.
test model에서 파일의 weight를 읽어와서 결과를 도출한다.

In [1]:
import tensorflow as tf

In [2]:
input_data = [[1, 5, 3, 7, 8, 10, 12],
              [5, 8, 10, 3, 9, 7, 1]]
label_data = [[0, 0, 0, 1, 0],
              [1, 0, 0, 0, 0]]

In [3]:
INPUT_SIZE = 7
HIDDEN1_SIZE = 10
HIDDEN2_SIZE = 8
CLASSES = 5
Learning_Rate = 0.5

In [4]:
x = tf.placeholder( tf.float32, shape=[None, INPUT_SIZE], name='x')
y_ = tf.placeholder( tf.float32, shape=[None, CLASSES], name='y_')

In [5]:
tensor_map = {x:input_data, y_:label_data}

In [6]:
# weight
w_h1 = tf.Variable(tf.truncated_normal(shape=[INPUT_SIZE, HIDDEN1_SIZE]), name='w_h1')
b_h1 = tf.Variable(tf.zeros([HIDDEN1_SIZE]), dtype=tf.float32, name='b_h1')

w_h2 = tf.Variable(tf.truncated_normal(shape=[HIDDEN1_SIZE, HIDDEN2_SIZE]), name='w_h2')
b_h2 = tf.Variable(tf.zeros([HIDDEN2_SIZE]), dtype=tf.float32, name='b_h2')

w_o = tf.Variable(tf.truncated_normal(shape=[HIDDEN2_SIZE, CLASSES]), name='w_o')
b_o = tf.Variable(tf.zeros([CLASSES]), dtype=tf.float32, name='b_o')

In [7]:
param_list = [w_h1, b_h1, w_h2, b_h2, w_o, b_o]
saver = tf.train.Saver(param_list)

variable이 끝난 시점에서 saver (그 이상으로 해버리면 모든 variable이 다 들어간다.)

In [8]:
# model
hidden1 = tf.sigmoid(tf.matmul(x, w_h1) + b_h1, name='hidden1')
hidden2 = tf.sigmoid(tf.matmul(hidden1, w_h2) + b_h2, name='hieen2')
y = tf.sigmoid(tf.matmul(hidden2, w_o) + b_o, name='y')

In [9]:
cost = tf.reduce_mean(tf.reduce_sum(-y_*tf.log(y)-(1-y_)*tf.log(1-y), reduction_indices=1))
train = tf.train.GradientDescentOptimizer(Learning_Rate).minimize(cost)

### **tf.reduce_sum**
![ex3_1](./image/3_1.png)

In [10]:
# insert line - 성능평가
accuray = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1)), tf.float32))

equal의 결과가 bool형 이므로 cast가 필요하다.  

보통 print할 때, save한다.
![ex3_2](./image/3_2.png)
저장할 때, 이름으로 저장한다.  
그래서 name을 parameter마다 지정해 줘야 나중에 데이터 가져올 때 이상이 없다.

In [11]:
sess = tf.Session()

In [12]:
init = tf.global_variables_initializer()
sess.run(init)

for i in range(100):
    _, loss, acc, result = sess.run([train, cost, accuray, y], tensor_map)
    
    if i % 10 == 0:
        saver.save(sess, "./checkpoint/my_model.ckpt")
        print("----------------------------")
        print("step : ", i)
        print("loss : ", loss)
        print("accuarcy : ", acc)
        print("result : ", result)

sess.close()

----------------------------
step :  0
loss :  2.9190667
accuarcy :  0.0
result :  [[0.4410853  0.5265158  0.4416273  0.2846872  0.09421104]
 [0.449956   0.5422097  0.51515424 0.14108446 0.10839973]]
----------------------------
step :  10
loss :  0.8125979
accuarcy :  1.0
result :  [[0.275751   0.08091093 0.03795361 0.71373224 0.05687724]
 [0.7084259  0.0354194  0.04556982 0.27659822 0.03197912]]
----------------------------
step :  20
loss :  0.47230166
accuarcy :  1.0
result :  [[0.17300154 0.0482968  0.02247761 0.81070226 0.0349617 ]
 [0.83011454 0.01716439 0.02646399 0.17155929 0.01826849]]
----------------------------
step :  30
loss :  0.30851692
accuarcy :  1.0
result :  [[0.11758213 0.03277746 0.01612057 0.8694012  0.02426067]
 [0.89036816 0.01100378 0.01887351 0.11182322 0.01294737]]
----------------------------
step :  40
loss :  0.220801
accuarcy :  1.0
result :  [[0.08579205 0.02403701 0.01242974 0.90442306 0.01824321]
 [0.9222202  0.00798652 0.01451566 0.07915518 0.010070